<a href="https://colab.research.google.com/github/huberyhub/deep_learning_ait/blob/main/5_AIT_Deep_learning_Exercise_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Copyright

<PRE>
Copyright (c) 2023 Bálint Gyires-Tóth - All Rights Reserved
You may use and modify this code for research and development purpuses.
Use this code for educational purposes (self-paced or instructor led) without the permission of the author is prohibited.
</PRE>



# California housing prices - modeling
In this notebook we will:
* using California housing prices data (https://www.kaggle.com/camnugent/california-housing-prices)
* exploring the data
* creating a model
* training the model
* evaluation on test data
* making predictions and regression plot

Exercises:
* please complete TODO parts


In [32]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.datasets import fetch_california_housing

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


In [33]:
# load California Housing Prices dataset
housing = fetch_california_housing()
X,Y = housing['data'], housing['target']
nb_samples = X.shape[0]

In [34]:
X.shape, Y.shape

((20640, 8), (20640,))

Splitting data into training, validation and test sets:

In [35]:
valid_split = 0.2
test_split  = 0.1

# calculate the validation and test split points
v_point = int(nb_samples * (1 - valid_split - test_split))
t_point = int(nb_samples * (1 - test_split))

X_train = X[:v_point]
Y_train = Y[:v_point]
X_valid = X[v_point:t_point]
Y_valid = Y[v_point:t_point]
X_test  = X[t_point:]
Y_test  = Y[t_point:]
# print out some of the data to check
print("X_train: ", X_train[:5])
print("Y_train: ", Y_train[:5])
print("X_valid: ", X_valid[:5])
print("Y_valid: ", Y_valid[:5])
print("X_test: ", X_test[:5])
print("Y_test: ", Y_test[:5])


X_train:  [[ 8.32520000e+00  4.10000000e+01  6.98412698e+00  1.02380952e+00
   3.22000000e+02  2.55555556e+00  3.78800000e+01 -1.22230000e+02]
 [ 8.30140000e+00  2.10000000e+01  6.23813708e+00  9.71880492e-01
   2.40100000e+03  2.10984183e+00  3.78600000e+01 -1.22220000e+02]
 [ 7.25740000e+00  5.20000000e+01  8.28813559e+00  1.07344633e+00
   4.96000000e+02  2.80225989e+00  3.78500000e+01 -1.22240000e+02]
 [ 5.64310000e+00  5.20000000e+01  5.81735160e+00  1.07305936e+00
   5.58000000e+02  2.54794521e+00  3.78500000e+01 -1.22250000e+02]
 [ 3.84620000e+00  5.20000000e+01  6.28185328e+00  1.08108108e+00
   5.65000000e+02  2.18146718e+00  3.78500000e+01 -1.22250000e+02]]
Y_train:  [4.526 3.585 3.521 3.413 3.422]
X_valid:  [[   3.744        30.            4.20348837    1.11918605  650.
     1.88953488   32.8        -117.26      ]
 [   5.4103       33.            5.27574751    0.96013289  683.
     2.26910299   32.81       -117.24      ]
 [   6.5881       34.            6.17346939    0.99744

Standardization of the data:

In [36]:
mean    = np.mean(X_train, axis=0)
std     = np.std(X_train, axis=0)
X_train = (X_train - mean) / std
X_valid = (X_valid - mean) / std
X_test  = (X_test - mean) / std

print("Mean values of the training, validation and test input data:", X_train.mean(), X_valid.mean(), X_test.mean())
print("Standard deviation of the training, validation and test input data:", X_train.std(), X_valid.std(), X_test.std())

Mean values of the training, validation and test input data: -8.41949864078967e-16 -0.033829980667358275 -0.05498069244049446
Standard deviation of the training, validation and test input data: 1.0 1.0511230319365579 1.998856651869891


Model definition, hints:
* one or two hidden layers and 8-64 neurons / layer
* the hidden layers should have non-linear activation functions
* the input and output layers' sizes are defined by the data
* as the output is continuous, it can be a linear layer

In [37]:
# Define the model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

Inspect the model summary:

In [38]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                576       
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 4801 (18.75 KB)
Trainable params: 4801 (18.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Number of trainable parameters in the model: TODO

Compiling the model, hints:
* the loss function has to match the task (regession)
* you should try ADAM and/or SGD optimizer

In [39]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

Defining the earlystopping callback function:

In [40]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(
    monitor='val_loss',
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

Train the model, use training and validation data and apply early stopping:

In [44]:
model.fit(X_train, Y_train,
        validation_data=(X_valid,Y_valid),
        batch_size=100,
        epochs=1000000,
        verbose=1,
        callbacks=[es])

Epoch 1/1000000
145/145 [==============================] - 0s 888us/step - loss: 0.2364 - accuracy: 0.0034 - val_loss: 0.6247 - val_accuracy: 0.0015
Epoch 2/1000000
145/145 [==============================] - 0s 546us/step - loss: 0.2323 - accuracy: 0.0034 - val_loss: 0.6138 - val_accuracy: 0.0015
Epoch 3/1000000
145/145 [==============================] - 0s 541us/step - loss: 0.2318 - accuracy: 0.0034 - val_loss: 0.5917 - val_accuracy: 0.0015
Epoch 4/1000000
145/145 [==============================] - 0s 536us/step - loss: 0.2314 - accuracy: 0.0034 - val_loss: 0.5897 - val_accuracy: 0.0015
Epoch 5/1000000
145/145 [==============================] - 0s 532us/step - loss: 0.2305 - accuracy: 0.0034 - val_loss: 0.5923 - val_accuracy: 0.0015
Epoch 6/1000000
145/145 [==============================] - 0s 526us/step - loss: 0.2303 - accuracy: 0.0034 - val_loss: 0.6074 - val_accuracy: 0.0015
Epoch 7/1000000
145/145 [==============================] - 0s 530us/step - loss: 0.2298 - accuracy: 0.0034

Evaluate the model on the test data:

In [42]:
model.evaluate(<TODO>)

SyntaxError: invalid syntax (939675598.py, line 1)

Run inference on the test data:

In [ ]:
preds = model.predict(<TODO>)

Inspect the result on a regression plot:

In [ ]:
plt.figure(figsize=(10,10))
sns.regplot(x=Y_test.reshape(-1), y=preds.reshape(-1)).set(xlim=(0,5),ylim=(0,5));